# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [2]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

In [3]:
# load messages dataset
messages = pd.read_csv("filepath_messages")
print(messages.shape)
print(pd.isnull(messages).any())
sum(pd.isnull(messages).any(axis = 1))


(26248, 4)
id          False
message     False
original     True
genre       False
dtype: bool


16064

In [4]:
#More than half of the rows do not contain any value for the "orignial" column --> drop it
messages = messages.drop(["original"],axis=1)
print(messages.shape)
print(messages)

(26248, 3)
          id                                            message   genre
0          2  Weather update - a cold front from Cuba that c...  direct
1          7            Is the Hurricane over or is it not over  direct
2          8                    Looking for someone but no name  direct
3          9  UN reports Leogane 80-90 destroyed. Only Hospi...  direct
4         12  says: west side of Haiti, rest of the country ...  direct
...      ...                                                ...     ...
26243  30261  The training demonstrated how to enhance micro...    news
26244  30262  A suitable candidate has been selected and OCH...    news
26245  30263  Proshika, operating in Cox's Bazar municipalit...    news
26246  30264  Some 2,000 women protesting against the conduc...    news
26247  30265  A radical shift in thinking came about as a re...    news

[26248 rows x 3 columns]


In [5]:
# load categories dataset
categories = pd.read_csv("filepath_categories")

print(categories.shape)
print(categories)

(26248, 2)
          id                                         categories
0          2  related-1;request-0;offer-0;aid_related-0;medi...
1          7  related-1;request-0;offer-0;aid_related-1;medi...
2          8  related-1;request-0;offer-0;aid_related-0;medi...
3          9  related-1;request-1;offer-0;aid_related-1;medi...
4         12  related-1;request-0;offer-0;aid_related-0;medi...
...      ...                                                ...
26243  30261  related-0;request-0;offer-0;aid_related-0;medi...
26244  30262  related-0;request-0;offer-0;aid_related-0;medi...
26245  30263  related-1;request-0;offer-0;aid_related-0;medi...
26246  30264  related-1;request-0;offer-0;aid_related-1;medi...
26247  30265  related-1;request-0;offer-0;aid_related-0;medi...

[26248 rows x 2 columns]


### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [6]:
# merge datasets
check = categories["id"]-messages["id"]
print(sum(check))
df = pd.concat([categories, messages], axis = 1).drop(columns=["id"])
print(pd.isnull(df).any())
print(df)

0
categories    False
message       False
genre         False
dtype: bool
                                              categories  \
0      related-1;request-0;offer-0;aid_related-0;medi...   
1      related-1;request-0;offer-0;aid_related-1;medi...   
2      related-1;request-0;offer-0;aid_related-0;medi...   
3      related-1;request-1;offer-0;aid_related-1;medi...   
4      related-1;request-0;offer-0;aid_related-0;medi...   
...                                                  ...   
26243  related-0;request-0;offer-0;aid_related-0;medi...   
26244  related-0;request-0;offer-0;aid_related-0;medi...   
26245  related-1;request-0;offer-0;aid_related-0;medi...   
26246  related-1;request-0;offer-0;aid_related-1;medi...   
26247  related-1;request-0;offer-0;aid_related-0;medi...   

                                                 message   genre  
0      Weather update - a cold front from Cuba that c...  direct  
1                Is the Hurricane over or is it not over  direct  
2   

### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [7]:
# create a dataframe of the 36 individual category columns
categories_splitted = categories['categories'].str.split(';', expand = True)
categories_splitted.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


In [8]:
# select the first row of the categories dataframe
row = categories_splitted.iloc[0]

# use this row to extract a list of new column names for categories.
# one way is to apply a lambda function that takes everything 
# up to the second to last character of each string with slicing
category_colnames = row.str[:-2]
print(category_colnames)

0                    related
1                    request
2                      offer
3                aid_related
4               medical_help
5           medical_products
6          search_and_rescue
7                   security
8                   military
9                child_alone
10                     water
11                      food
12                   shelter
13                  clothing
14                     money
15            missing_people
16                  refugees
17                     death
18                 other_aid
19    infrastructure_related
20                 transport
21                 buildings
22               electricity
23                     tools
24                 hospitals
25                     shops
26               aid_centers
27      other_infrastructure
28           weather_related
29                    floods
30                     storm
31                      fire
32                earthquake
33                      cold
34            

In [9]:
# rename the columns of `categories`
categories_splitted.columns = category_colnames
categories_splitted.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


### 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. You may need to first convert the Series to be of type string, which you can do with `astype(str)`.

In [10]:
for column in categories_splitted:
    # set each value to be the last character of the string
    categories_splitted[column] = categories_splitted[column].str[-1]
    
    # convert column from string to numeric
    categories_splitted[column] = pd.to_numeric(categories_splitted[column])
categories_splitted.head()
print(categories_splitted.shape)
print(df.shape)

(26248, 36)
(26248, 3)


### 5. Replace `categories` column in `df` with new category columns.
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [11]:
# drop the original categories column from `df`

df = df.drop(columns=["categories"])

df.head(-100)

,message,genre
0,Weather update - a cold front from Cuba that c...,direct
1,Is the Hurricane over or is it not over,direct
2,Looking for someone but no name,direct
3,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,"says: west side of Haiti, rest of the country ...",direct
...,...,...
26143,Officials said they would begin taking a censu...,news
26144,"Despite their territorial losses, the insurgen...",news
26145,Other essential items distributed by ADRA incl...,news
26146,The Philippines announced Wednesday it was dis...,news


In [12]:
# concatenate the original dataframe with the new `categories` dataframe
df.reset_index(drop=True, inplace=True)
categories.reset_index(drop=True, inplace=True)
df = pd.concat([df,categories_splitted], axis = 1)
print(df)

                                                 message   genre  related  \
0      Weather update - a cold front from Cuba that c...  direct        1   
1                Is the Hurricane over or is it not over  direct        1   
2                        Looking for someone but no name  direct        1   
3      UN reports Leogane 80-90 destroyed. Only Hospi...  direct        1   
4      says: west side of Haiti, rest of the country ...  direct        1   
...                                                  ...     ...      ...   
26243  The training demonstrated how to enhance micro...    news        0   
26244  A suitable candidate has been selected and OCH...    news        0   
26245  Proshika, operating in Cox's Bazar municipalit...    news        1   
26246  Some 2,000 women protesting against the conduc...    news        1   
26247  A radical shift in thinking came about as a re...    news        1   

       request  offer  aid_related  medical_help  medical_products  \
0    

### 6. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [13]:
# check number of duplicates
duplicate_counts = df.duplicated().sum()
print(duplicate_counts)

33


In [14]:
# drop duplicates
df.drop_duplicates(inplace=True)

In [15]:
# check number of duplicates
duplicate_counts = df.duplicated().sum()
print(duplicate_counts)

0


### 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [16]:
engine = create_engine('sqlite:///Disaster_Response.db')
df.to_sql('df', engine, index=False)

26215

### 8. Use this notebook to complete `process_data.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `process_data.py` in the classroom on the `Project Workspace IDE` coming later.